In [33]:
import arcpy
from arcpy.sa import Slope

# Set environment settings
arcpy.env.workspace = r"C:\LiDAR_2"
arcpy.env.overwriteOutput = True

""""
This code is intended to help with my research. I get from drone flights .laz LiDAR point clouds and the idea is to come
up with a DEM to perform ground analysis. The idea is to convert from .laz to .las to work it in ArcGIS Pro, then, create
a LAS Dataset to perform point cloud analysis. Once the LAS Dataset is created, classify the ground to obtain ground vs
non-ground points to extract just the terrain and, with that, come up with a DEM.
From that point I am not entirely sure which path to take. I think that I will create a slope map to obtain the more critical
areas or to try to extract the river channel from the DEM, I will think what will be most useful for my research
"""

# Input and output paths
input_laz = r"C:\LiDAR_2\2024\Lone_Oaks_2024-05-01_Raw.copc.laz" #takes the point cloud as an input as .laz
converted_las = r"C:\LiDAR_2\project_output\\" #to store the .las from .laz
las_dataset = r"C:\LiDAR_2\lasdataset\dataset.lasd" #to store .lasd dataset
output_raster = r"C:\LiDAR_2\raster\dem.tif" #to store DEM raster
slope_model = r"C:\LiDAR_2\raster\slope.tif" #to store slope model

# NAD 83 State Plane Tennessee FIPS 4100 (Feet)
spatial_reference = arcpy.SpatialReference(2274)  # EPSG Code for NAD 1983 Tennessee State Plane 

#convert LAZ to LAS
arcpy.conversion.ConvertLas(
    input_laz,
    converted_las,
    "1.4"# LAS file version
    "NO_COMPRESSION"
)

#create LAS Dataset
arcpy.management.CreateLasDataset(
    [r"C:\LiDAR_2\project_output\Lone_Oaks_2024-05-01_Raw.copc.las"],  #takes the las file
    las_dataset,
    "RECURSION", #to loop through all files in the folder
    None, #not sure why, but after many attempts this worked
    spatial_reference 
)


#classify Ground
arcpy.ddd.ClassifyLasGround(
    in_las_dataset = las_dataset,
    method = "CONSERVATIVE", #method to classify ground to non-ground points
    dem_resolution = 5, #resolution for sampling
)


#export LAS Dataset to Raster
arcpy.conversion.LasDatasetToRaster(
    in_las_dataset=las_dataset,
    out_raster = output_raster,
    value_field="ELEVATION",  # Field to base raster values on
    interpolation_type="BINNING MAXIMUM NATURAL_NEIGHBOR", #method to interpolate the point cloud to raster
    data_type="FLOAT", #type for the cell
    sampling_type="CELLSIZE",
    sampling_value=3  #cell size
)


print("Process completed successfully")
"""
Until this point I am able to create the DEM, but despite the point cloud is classified, it is not exporting the ground points.
It is not a matter of wrong classification but rather a matter of a wrong selection. I need to see how to select those features and export
just the ground. Once the DEM is created I will probably concenctrate on riverbank analysis and I will try to see if it is easier to extract
the river and do a slope analysis on the river bank or hand sketch the river course to then analyise river bank slope.
""""


'\n# Step 5: Create Slope Model\narcpy.CheckOutExtension("Spatial")  # Check out Spatial Analyst extension\nslope = Slope(output_raster, output_measurement="DEGREE")  # Calculate slope\nslope.save(slope_model)  # Save the slope model\narcpy.CheckInExtension("Spatial")  # Check in Spatial Analyst extension\n\nprint("Process completed successfully!")\n'